In [11]:
import sys
from torch import nn
from torch.utils.data import DataLoader 
from nnunet.training.network_training.nnUNetTrainerV2 import nnUNetTrainerV2


sys.path.append("../")
from dataset import CalgaryCampinasDataset, ACDCDataset, MNMDataset
from model.unet import UNet2D
from trainer.unet_trainer import UNetTrainerCalgary, UNetTrainerACDC
from utils import volume_collate
from losses import DiceScoreMMS, CrossEntropyTargetArgmax, DiceScoreCalgary, SurfaceDiceCalgary

In [15]:
cfg = {
    'debug': True,
    'log': False,
    'description': 'calgary_unet-test',
    'project': 'IPMI2023',
    
    # Data params
    'n': 0,
    'root': '../../',
    'data_path': 'data/conp-dataset/projects/calgary-campinas/CC359/Reconstructed/',
    'train_site': 6,
    'augment': False,
    'validation': True,
    'channel_out': 8,
    
    # Training params
    'batch_size': 32,
    'lr': 1e-3,
    'epochs': 2,
    'patience': 4
}

description = cfg['description'] + str(cfg['n'])
print(f"Decsription: {description}")

data_path = cfg['root'] + cfg['data_path']

train_set = CalgaryCampinasDataset(data_path=data_path, site=cfg['train_site'], 
                                   augment=cfg['augment'], normalize=True, 
                                   split='train', debug=cfg['debug'])
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=False, num_workers=10)


valid_set = CalgaryCampinasDataset(data_path=data_path, site=cfg['train_site'], 
                                   normalize=True, volume_wise=True,
                                   split='validation')
valid_loader = DataLoader(valid_set, batch_size=1, shuffle=False, drop_last=False, 
                      collate_fn=volume_collate)


# UNET
seg_model   = UNet2D(n_chans_in=1, n_chans_out=1, n_filters_init=cfg['channel_out'])
criterion   = nn.BCEWithLogitsLoss()
eval_metrics = {
    "Volumetric Dice": DiceScoreCalgary(),
    "Surface Dice": SurfaceDiceCalgary()
}
root = cfg['root']
unet_trainer = UNetTrainerCalgary(model=seg_model, 
                                  criterion=criterion, 
                                  train_loader=train_loader, 
                                  valid_loader=valid_loader, 
                                  root=root, 
                                  eval_metrics=eval_metrics, 
                                  lr=cfg['lr'],
                                  n_epochs=cfg['epochs'],
                                  description=description,
                                  patience=cfg['patience'],
                                  log=False)

unet_trainer.fit()

Decsription: calgary_unet-test0


  0%|          | 0/2 [00:00<?, ?it/s]

No metrics found


In [3]:
### Dataloader
## Initialize trainer to get data loaders with data augmentations from training
nnUnet_prefix = '../../../nnUNet/'
pkl_file          = nnUnet_prefix + 'data/nnUNet_preprocessed/Task500_ACDC/nnUNetPlansv2.1_plans_2D.pkl'
fold              = 0
output_folder     = nnUnet_prefix + 'results/nnUnet/nnUNet/2d/Task027_ACDC/nnUNetTrainerV2__nnUNetPlansv2.1/'
dataset_directory = nnUnet_prefix + 'data/nnUNet_preprocessed/Task500_ACDC'

trainer = nnUNetTrainerV2(pkl_file, 0, output_folder, dataset_directory)
trainer.initialize()

train_loader = trainer.tr_gen
valid_loader = trainer.val_gen


cfg = {
    'debug': True,
    'log': False,
    'description': f'acdc_unet_test',
    'project': 'MICCAI2023',

    # Data params
    'n': 0,
    'root': '../../',
    'validation': True,
    'channel_out': 8,

    # Training params
    'batch_size': 32,
    'lr': 1e-3,
    'epochs': 2,
    'patience': 8
}


seg_model    = UNet2D(n_chans_in=1, n_chans_out=4, n_filters_init=cfg['channel_out'])
criterion    = CrossEntropyTargetArgmax()
eval_metrics = {
    "Volumetric Dice": DiceScoreMMS()
} 
root = cfg['root']
unet_trainer = UNetTrainerACDC(model=seg_model,
                               criterion=criterion,
                               train_loader=train_loader,
                               valid_loader=valid_loader,
                               num_batches_per_epoch=trainer.num_batches_per_epoch,
                               num_val_batches_per_epoch=trainer.num_val_batches_per_epoch,
                               root=root,
                               eval_metrics=eval_metrics,
                               lr=cfg['lr'],
                               n_epochs=cfg['epochs'],
                               description=cfg["description"],
                               patience=cfg['patience'],
                               log=cfg['log'])

unet_trainer.fit()

loading dataset
loading all case properties
2023-06-23 11:32:21.063044: Using splits from existing split file: ../../../nnUNet/data/nnUNet_preprocessed/Task500_ACDC/splits_final.pkl
2023-06-23 11:32:21.064343: The split file contains 5 splits.
2023-06-23 11:32:21.064464: Desired fold for training: 0
2023-06-23 11:32:21.064769: This split has 160 training and 40 validation cases.
unpacking dataset
done


  0%|          | 0/2 [00:00<?, ?it/s]

using pin_memory on device 0
using pin_memory on device 0
No metrics found
